<a href="https://colab.research.google.com/github/acmilannesta/Bert-embedding/blob/master/Bert_adultreadmission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install keras_bert

In [0]:
import json
import numpy as np
import pandas as pd
from random import choice
from keras_bert import load_trained_model_from_checkpoint, Tokenizer, AdamWarmup, calc_train_steps
import re, os, gc
import codecs
from keras.layers import *
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings("ignore")

In [0]:
MAXLEN = 300
BATCH_SIZE = 8
config_path = '/content/drive/My Drive/biobert_pretrain_output_all_notes_150000/bert_config.json' #'uncased_L-12_H-768_A-12/bert_config.json'
checkpoint_path = '/content/drive/My Drive/biobert_pretrain_output_all_notes_150000/model.ckpt' #'uncased_L-12_H-768_A-12/bert_model.ckpt'
dict_path = '/content/drive/My Drive/biobert_pretrain_output_all_notes_150000/vocab.txt' #'uncased_L-12_H-768_A-12/vocab.txt'

In [0]:
token_dict = {}
with codecs.open(dict_path, 'r', 'utf8') as reader:
  for line in reader:
      token = line.strip()
      token_dict[token] = len(token_dict)

In [0]:
# class OurTokenizer(Tokenizer):
#     def _tokenize(self, text):
#         R = []
#         for c in text:
#             if c in self._token_dict:
#                 R.append(c)
#             # elif self._is_space(c):
#             #     R.append('[unused1]')
#             else:
#                 R.append('[UNK]')
#         return R

tokenizer = Tokenizer(token_dict)

In [0]:
data = pd.read_csv('drive/My Drive/Adult readmission/Combined_vars_04092019_notes.csv')
data.CODE_REHOSP = np.where(data.CODE_REHOSP==2, 0, 1)
data1 = data[['sw_note', 'CODE_REHOSP']].dropna()
data1['sw_note'] = data1['sw_note'].apply(lambda x: x.replace('\r', ''))
X_train, X_test, y_train, y_test = train_test_split(data1['sw_note'], data1['CODE_REHOSP'], test_size=0.2, random_state=0, stratify=data1['CODE_REHOSP'])
train_data = list(zip(X_train, y_train))
valid_data = list(zip(X_test, y_test))

In [0]:
data1.to_csv('drive/My Drive/Data/sw_note.csv', index=False)
# data1.shape

In [0]:
def seq_padding(X, padding=0):
  L = [len(x) for x in X]
  ML = max(L)
  return np.array([np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X])

class data_generator:
  def __init__(self, data, batch_size=BATCH_SIZE, maxlen=MAXLEN):
    self.data = data
    self.batch_size = batch_size
    self.maxlen = maxlen
    self.steps = len(self.data) // self.batch_size
    if len(self.data) % self.batch_size != 0:
        self.steps += 1
  def __len__(self):
    return self.steps
  def __iter__(self):
    while True:
      idxs = list(range(len(self.data)))
      np.random.shuffle(idxs)
      X1, X2, Y = [], [], []
      for i in idxs:
        d = self.data[i]
        text = d[0][:self.maxlen]
        x1, x2 = tokenizer.encode(first=text)
        y = d[1]
        X1.append(x1)
        X2.append(x2)
        Y.append([y])
        if len(X1) == self.batch_size or i == idxs[-1]:
          X1 = seq_padding(X1)
          X2 = seq_padding(X2)
          Y = seq_padding(Y)
          yield [X1, X2], Y
          [X1, X2, Y] = [], [], []

In [0]:
bert_model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    trainable=True)

x1_in = Input(shape=(None,))
x2_in = Input(shape=(None,))

x = bert_model([x1_in, x2_in])
x = Lambda(lambda x: x[:, 0])(x)
p = Dense(1, activation='sigmoid')(x)

model = Model([x1_in, x2_in], p)
# def auroc(y_true, y_pred):
#     return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)
decay_steps, warmup_steps = calc_train_steps(
    len(train_data),
    batch_size=BATCH_SIZE,
    epochs=2,
)

model.compile(
    loss='binary_crossentropy',
    # optimizer=Adam(1e-5),
    optimizer=AdamWarmup(decay_steps=decay_steps, warmup_steps=warmup_steps, lr=1e-5),
    metrics=['acc']
)

In [64]:
train_D = data_generator(train_data)
valid_D = data_generator(valid_data)
model.fit_generator(
    train_D.__iter__(),
    steps_per_epoch=len(train_D),
    epochs=2,
    validation_data=valid_D.__iter__(),
    validation_steps=len(valid_D)
)

Epoch 1/2
114/114 [==============================] - 61s 536ms/step - loss: 0.6189 - acc: 0.7039 - val_loss: 0.6115 - val_acc: 0.7061
Epoch 2/2
114/114 [==============================] - 28s 245ms/step - loss: 0.5773 - acc: 0.7057 - val_loss: 0.6194 - val_acc: 0.7061


In [0]:
pred = model.predict_generator(valid_D.__iter__(), len(valid_D))

In [66]:
print('ROC-AUC: {:.2f}%'.format(roc_auc_score([x[1] for x in valid_data], pred)*100))
print('Accuracy Score: {:.2f}%'.format(accuracy_score([x[1] for x in valid_data], [int(x>0.5) for x in pred])*100))
print('Precision Score: {:.2f}%'.format(precision_score([x[1] for x in valid_data], [int(x>0.5) for x in pred])*100))
print('Recall Score: {:.2f}%'.format(recall_score([x[1] for x in valid_data], [int(x>0.5) for x in pred])*100))
print('F1 Score: {:.2f}%'.format(f1_score([x[1] for x in valid_data], [int(x>0.5) for x in pred])*100))

ROC-AUC: 49.98%
Accuracy Score: 70.61%
Precision Score: 0.00%
Recall Score: 0.00%
F1 Score: 0.00%
